<img style="float: left" src="images/spark.png" />
<img style="float: right" src="images/surfsara.png" />
<hr style="clear: both" />

# Counting kmers with Apache Spark

In our second notebook we will look at genomics data and count the numer of occurences of specific subsequences in the reads.

_You can edit the cells below and execute the code by selecting the cell and press Shift-Enter. Code completion is supported by use of the Tab key._

During the exercises you may want to refer to the [PySpark documentation](https://spark.apache.org/docs/1.6.1/api/python/pyspark.html#pyspark.RDD) for more information on possible transformations and actions.

In [ ]:
# initialize Spark
from pyspark import SparkContext, SparkConf

if not 'sc' in globals(): # This Python 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')
    sc = SparkContext(conf=conf)

##  RDD from a FASTA file

We will start by creating an RDD from a FASTA file. We will read this using the `sc.textFile` method we also used in the first notebook.

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

reads = sc.textFile("file:////home/jovyan/work/data/blast/input/CAM_SMPL_GS108.fa")

pp.pprint(reads.take(3))

We see that we are able to read the FASTA file, but the division in records is incorrect. The metadata and sequence data of a single read are now split over two separate records. The clean solution to this would be to write a custom InputFormat in Java or Scala. For simplicity in this tutorial we will do some text munching in Spark to get the same effect.

## Remove metadata

In this case we are not interested in the metadata. We select only the sequence data by adding an index number to all the records and use a [`filter`](https://spark.apache.org/docs/1.6.1/api/python/pyspark.html#pyspark.RDD.filter) to select only the odd-numberd records.

In [ ]:
indexed_reads = reads.zipWithIndex()

pp.pprint(indexed_reads.take(2))

In [ ]:
# Create an RDD 'indexed_sequences' from' indexed_reads' that only contains the records with an odd index.

### BEGIN SOLUTION
indexed_sequences = indexed_reads.filter(lambda x: x[1] % 2 == 1)
### END SOLUTION

sequences = indexed_sequences.keys().cache()

pp.pprint(sequences.take(2))

## Extending WordCount: BaseCount

Now that we have our input data in the format we want we can start to do something with it. Our first exercise is a variation on the WordCount we did in the first notebook. This time we will not count words, but the individual bases in the sequences.

Hint: the easiest way in Python to split a string `s` into characters is: `list(s)`.

In [ ]:
# Create an RDD 'bases' from 'sequences' containing all the individual bases (letters)

### BEGIN SOLUTION
bases = sequences.flatMap(lambda s: list(s))
### END SOLUTION 

pp.pprint(bases.take(5))

Make key-value pairs and sum using reduceByKey

In [ ]:
# Create an RDD 'basecount' 'from bases' with (base, #occurences) pairs

### BEGIN SOLUTION
basecounts = bases.map(lambda b: (b, 1)).reduceByKey(lambda a,b: a + b)
### END SOLUTION

Since we know the number of records in this RDD is very small (5), it is safe to call `collect` on the RDD and print the results.

In [ ]:
pp.pprint(basecounts.collect())

## Extending BaseCount: KmerCount

Lets make it a bit more interesting.

Select all substrings of length k. We have written a helper function for you.

In [ ]:
def sliding(seq, size):
    result = []
    for i in range(0, len(seq) - size + 1):
        result.append(seq[i:i + size])
    return result

sliding("GAGATCTCCTGTGGTGTCCTTGGTCATAGTGATTTGCTCCTACAA", 5)

Create an RDD with all the subsequences of length 21.

In [ ]:
### BEGIN SOLUTION
kmers = sequences.flatMap(lambda s: sliding(s, 21))
### END SOLUTION

In [ ]:
### BEGIN SOLUTION
kmercounts = kmers.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x + y)
### END SOLUTION
kmercounts.cache()

Use takeOrdered to get the 10 most frequent 21-mers

In [ ]:
### BEGIN SOLUTION
top10 = kmercounts.takeOrdered(10, key=lambda x: -x[1])
### END SOLUTION
pp.pprint(top10)

Finally, we look at the distribution of 21-mers. x-axis the number of occurences of a 21-mer, y-axis the number of 21-mers that occur x times.

In [ ]:
### BEGIN SOLUTION
kmerdist = kmercounts.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)
### END SOLUTION 

In [ ]:
kmerdistsorted = kmerdist.map(lambda x: (x[1], x[0])).sortByKey(True).cache()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

x = kmerdistsorted.keys().collect()
y = kmerdistsorted.values().collect()

plt.plot(x, y)
plt.yscale('log')
plt.title("kmer distribution")
plt.xlabel("kmer matches")
plt.ylabel("genome wide frequency")
plt.legend()
plt.show()